In [1]:
import re
import json
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from itertools import chain
from collections import Counter

In [2]:
tqdm.pandas()

In [3]:
pd.options.display.max_rows = 10

In [4]:
def read_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [5]:
d_199 = read_pickle("../data/supports/parsed_199.pkl")

In [6]:
d_6725 = read_pickle("../data/supports/parsed_7003.pkl")

In [7]:
## ada 43 screen_names yang overlap
d_6725[d_6725.screen_name.isin(d_199.screen_name.unique())].screen_name.unique()

array(['AirinAirin_NZ', 'ariel_heryanto', 'basuki_btp', 'BNPB_Indonesia',
       'BobbySurya85', 'budimandjatmiko', 'BukuMojok', 'cokmaret',
       'D4tuk_T4mburin', 'Dennysiregar7', 'fadlizon', 'fahiraidris',
       'FaldoMaldini', 'fatih_aqsa', 'FiersaBesari', 'GaekParewa1',
       'ganjarpranowo', 'gjynmmnggllagi', 'guh_guhnan02', 'HamdJR787',
       'JayMerkano2', 'JDAgraria', 'jr_kw19', 'kafiradikalis',
       'KawalCOVID19', 'kurawa', 'lawanpatriarki', 'mazzini_gsp',
       'Namaku_Izrail', 'narkosun', 'pangeransiahaan', 'PolitikusBadut',
       'Prof_Kupie', 'RachlanNashidik', 'RidwanS38612261',
       'rockygerung_rg', 'RyanDirhamsyah', 'SabangIna', 'sigitwid',
       'TsamaraDKI', 'urmila__011', 'yunartowijaya', 'yusuf_dumdum'],
      dtype=object)

In [22]:
d_tweets = pd.concat([d_199, d_6725], axis = 0, sort=False)

In [24]:
## remove duplicates
d_tweets.drop_duplicates(subset=["id_tweet"], keep='first', inplace=True)

In [27]:
d_tweets.reset_index(drop=True, inplace=True)

In [28]:
d_tweets.columns

Index(['screen_name', 'id_tweet', 'full_text', 'hashtags', 'user_mentions',
       'created_at', 'quote_is_quote_status', 'quote_screen_name',
       'quote_id_tweet', 'quote_full_text', 'quote_hashtags',
       'quote_user_mentions', 'quote_created_at', 'in_reply_to_status_id_str',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name'],
      dtype='object')

In [29]:
column_ordered = ['screen_name', 'id_tweet', 'full_text', 'hashtags', 'user_mentions', 'created_at',
                 'quote_is_quote_status', 'quote_screen_name', 'quote_id_tweet', 'quote_full_text',
                  'quote_hashtags', 'quote_user_mentions', 'quote_created_at', 
                  "in_reply_to_status_id_str", "in_reply_to_user_id_str", "in_reply_to_screen_name"]

In [30]:
d_tweets = d_tweets[column_ordered]

In [31]:
d_tweets

,screen_name,id_tweet,full_text,hashtags,user_mentions,created_at,quote_is_quote_status,quote_screen_name,quote_id_tweet,quote_full_text,quote_hashtags,quote_user_mentions,quote_created_at,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name
0,212Ujee2,1272955449160040449,#HapusRUUHIPDariMukaBumi \n#HapusRUUHIPDariMuk...,"[{'text': 'HapusRUUHIPDariMukaBumi', 'indices'...",[],Tue Jun 16 18:13:24 +0000 2020,True,Par3w4_Minang,1272901801516335112,Moncong putih ingin cuci tangan \n\nKata si Po...,[],[],Tue Jun 16 14:40:13 +0000 2020,NaN,NaN,NaN
1,212Ujee2,1272573148974571526,Sampah n benalu\nPerusak demokrasi wajib kita ...,"[{'text': 'BuzzeRpPublicEnemy', 'indices': [59...",[],Mon Jun 15 16:54:16 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,212Ujee2,1272242834725208064,Saatnya kita bersatu\nWahai umat Islam\n\n#Sto...,"[{'text': 'Stop_RUUHIP', 'indices': [39, 51]},...",[],Sun Jun 14 19:01:43 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,212Ujee2,1267860265694978049,Umat Islam\nHrs diam \nTerus sampai kapan?\nKe...,"[{'text': 'BalikinDanaHaji', 'indices': [104, ...",[],Tue Jun 02 16:46:57 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,212Ujee2,1266354000716107780,Hai Pemimpin !!!\nApakah udh ga ada lagi \nPro...,"[{'text': 'DirutBokep', 'indices': [204, 215]}...","[{'screen_name': '__p3jalan____', 'name': 'p3j...",Fri May 29 13:01:36 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983962,___denmasNg,1276777544377757696,@Izznaq Sepakat juga 😊👍🇮🇩\n\n#StopProvokasiFok...,"[{'text': 'StopProvokasiFokusCovid19', 'indice...","[{'screen_name': 'Izznaq', 'name': 'Izza', 'id...",Sat Jun 27 07:21:02 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1276738370522116097,2547120980,Izznaq
1983963,___denmasNg,1276777416355008513,@bringas3006 Tenggelam kan!\n\n#StopProvokasiF...,"[{'text': 'StopProvokasiFokusCovid19', 'indice...","[{'screen_name': 'bringas3006', 'name': 'Cebon...",Sat Jun 27 07:20:32 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1276738931241803778,859740055362981889,bringas3006
1983964,___denmasNg,1276777335975317504,@Kesit_ Ahay 😁\n\n#StopProvokasiFokusCovid19,"[{'text': 'StopProvokasiFokusCovid19', 'indice...","[{'screen_name': 'Kesit_', 'name': 'Keep Stron...",Sat Jun 27 07:20:12 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1276736247977914368,2478685009,Kesit_
1983965,___denmasNg,1276777259618021377,@IswantoBagaskar Siap laksanakan\n\n#StopProvo...,"[{'text': 'StopProvokasiFokusCovid19', 'indice...","[{'screen_name': 'IswantoBagaskar', 'name': 'I...",Sat Jun 27 07:19:54 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1276736866880897024,1670116434,IswantoBagaskar


In [32]:
## should change 1 to 0
d_user_mention_quote = d_tweets[(d_tweets.user_mentions.apply(len) > 0) | (d_tweets.quote_screen_name)]

In [33]:
def interaction(row):
    screen_name = row.screen_name
    user_mentions = row.user_mentions
    quote_screen_name = row.quote_screen_name
    user_interaction = []
    for username in user_mentions:
        mentioned_screen_name = username["screen_name"]
        user_interaction.append(screen_name + " - " + mentioned_screen_name)
        
    if isinstance(quote_screen_name, str):
        user_interaction.append(screen_name + " - " + quote_screen_name)
        
    return user_interaction

In [34]:
d_user_mention_quote.shape

(1532469, 16)

In [35]:
d_user_mention_quote

,screen_name,id_tweet,full_text,hashtags,user_mentions,created_at,quote_is_quote_status,quote_screen_name,quote_id_tweet,quote_full_text,quote_hashtags,quote_user_mentions,quote_created_at,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name
0,212Ujee2,1272955449160040449,#HapusRUUHIPDariMukaBumi \n#HapusRUUHIPDariMuk...,"[{'text': 'HapusRUUHIPDariMukaBumi', 'indices'...",[],Tue Jun 16 18:13:24 +0000 2020,True,Par3w4_Minang,1272901801516335112,Moncong putih ingin cuci tangan \n\nKata si Po...,[],[],Tue Jun 16 14:40:13 +0000 2020,NaN,NaN,NaN
4,212Ujee2,1266354000716107780,Hai Pemimpin !!!\nApakah udh ga ada lagi \nPro...,"[{'text': 'DirutBokep', 'indices': [204, 215]}...","[{'screen_name': '__p3jalan____', 'name': 'p3j...",Fri May 29 13:01:36 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,212Ujee2,1265626077054959616,"Nugie "" Tertipu "" https://t.co/hoV57tJjtT \n\...","[{'text': 'TelevisiVembelaRezimIndochina', 'in...","[{'screen_name': 'Nadine_Oliv', 'name': 'Nadin...",Wed May 27 12:49:05 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,212Ujee2,1259983627829440513,Cocok neh\nBerbicara Bohong\nDipercaya Khinat\...,"[{'text': 'TukangNgibulNaikTahta', 'indices': ...",[],Mon May 11 23:08:00 +0000 2020,True,eiznek_kenzie,1259909958029635584,Orang dusta pasti celaka\nBarakallahu gurunda ...,"[{'text': 'TukangNgibulNaikTahta', 'indices': ...",[],Mon May 11 18:15:16 +0000 2020,NaN,NaN,NaN
11,212Ujee2,1258823052793483264,#JengSriBalikinDuitDKI\n💪🇮🇩🇮🇩🇮🇩🇮🇩🇮🇩🇮🇩🇮🇩✊\n#Jen...,"[{'text': 'JengSriBalikinDuitDKI', 'indices': ...","[{'screen_name': 'Nadine_Oliv', 'name': 'Nadin...",Fri May 08 18:16:18 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983962,___denmasNg,1276777544377757696,@Izznaq Sepakat juga 😊👍🇮🇩\n\n#StopProvokasiFok...,"[{'text': 'StopProvokasiFokusCovid19', 'indice...","[{'screen_name': 'Izznaq', 'name': 'Izza', 'id...",Sat Jun 27 07:21:02 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1276738370522116097,2547120980,Izznaq
1983963,___denmasNg,1276777416355008513,@bringas3006 Tenggelam kan!\n\n#StopProvokasiF...,"[{'text': 'StopProvokasiFokusCovid19', 'indice...","[{'screen_name': 'bringas3006', 'name': 'Cebon...",Sat Jun 27 07:20:32 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1276738931241803778,859740055362981889,bringas3006
1983964,___denmasNg,1276777335975317504,@Kesit_ Ahay 😁\n\n#StopProvokasiFokusCovid19,"[{'text': 'StopProvokasiFokusCovid19', 'indice...","[{'screen_name': 'Kesit_', 'name': 'Keep Stron...",Sat Jun 27 07:20:12 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1276736247977914368,2478685009,Kesit_
1983965,___denmasNg,1276777259618021377,@IswantoBagaskar Siap laksanakan\n\n#StopProvo...,"[{'text': 'StopProvokasiFokusCovid19', 'indice...","[{'screen_name': 'IswantoBagaskar', 'name': 'I...",Sat Jun 27 07:19:54 +0000 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1276736866880897024,1670116434,IswantoBagaskar


In [36]:
d_user_mention_quote.loc[:, "interaction_list"] = d_user_mention_quote.progress_apply(interaction, axis = 1)

100%|█████████████████████████████████████████████████████████████████████| 1532469/1532469 [01:35<00:00, 16033.88it/s]
C:\Users\Andreas Chandra\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Andreas Chandra\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [37]:
list_of_interactions = d_user_mention_quote.loc[:, "interaction_list"].to_list()

In [38]:
interactions_count = Counter(list(chain(*list_of_interactions)))

In [39]:
d_connection = pd.DataFrame(data = {"connection": list(interactions_count.keys()), 
                                    "frequency": list(interactions_count.values())})

In [40]:
d_connection.shape

(944157, 2)

In [42]:
d_connection.head()

,connection,frequency
0,212Ujee2 - Par3w4_Minang,2
1,212Ujee2 - __p3jalan____,52
2,212Ujee2 - _garuda501,56
3,212Ujee2 - Kanseulir,84
4,212Ujee2 - EW1jaya,41


In [41]:
d_connection.to_csv("../data/supports/connection.csv", index=False)

## Preparing data for tweet network

In [43]:
d_tweet_connection = d_tweets[(~d_tweets.in_reply_to_status_id_str.isna()) | (~d_tweets.quote_id_tweet.isna())]

In [44]:
def tweet_interaction(df):
    id_tweet = df.id_tweet
    in_reply_to_status_id_str = df.in_reply_to_status_id_str
    quote_id_tweet = df.quote_id_tweet
    
    connection = []
    
    if isinstance(in_reply_to_status_id_str, str):
        connection.append({"id_tweets": (id_tweet, in_reply_to_status_id_str), 
                           "screen_name": (df.screen_name, df.in_reply_to_screen_name),
                           "type": "in_reply"})
    
    if isinstance(quote_id_tweet, str):
        connection.append({"id_tweets": (id_tweet, quote_id_tweet),
                           "screen_name": (df.screen_name, df.quote_screen_name),
                           "type": "in_quote"})
        
    return connection

In [45]:
d_tweet_connection.loc[:, "interaction_list"] = d_tweet_connection.progress_apply(tweet_interaction, axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 767853/767853 [00:43<00:00, 17533.34it/s]


In [46]:
id_tweet_interaction = list(chain(*d_tweet_connection.interaction_list.values.tolist()))

In [48]:
with open("../data/supports/id_tweet_connection.json", 'w') as f:
    json.dump(id_tweet_interaction, f)